# Parsing open sources

In [1]:
import re
import random
import pickle
import requests
from collections import defaultdict
from itertools import product
from urllib.parse import unquote

import psycopg2
import numpy as np
import pandas as pd
import scrapy
from scrapy.crawler import CrawlerProcess

from postgresql_client import PostgreSQLClient

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
plt.style.use('seaborn-poster')

/opt/jupyter/lib64/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


## Create link project_id -> project_title

In [2]:
%%time
client = PostgreSQLClient()
query = """
WITH object_ids AS (
    SELECT 
        DISTINCT object_id AS project_id
        --COUNT(DISTINCT object_id)
    FROM opossum.person_object_relationship_vw
    WHERE 1=1
        AND object_type = 'project'
)
SELECT 
    projects.project_id,
    projects.project_title
FROM opossum.project_vw AS projects
INNER JOIN object_ids ON 1=1
    AND projects.project_id = object_ids.project_id
"""
project_id_title = client.get_data(query)
project_id_title = pd.DataFrame(project_id_title, columns=['project_id', 'project_title'])
print(project_id_title.shape)

(198, 2)
CPU times: user 2.81 ms, sys: 1.73 ms, total: 4.54 ms
Wall time: 222 ms


In [3]:
search_str = 'https://www.kinopoisk.ru/index.php?kp_query='
project_id_title['search_query'] = project_id_title['project_title'].apply(lambda x: search_str+x)

In [15]:
project_id_title.iloc[156]

project_id                                                     674
project_title                                  НЕфутбольная страна
search_query     https://www.kinopoisk.ru/index.php?kp_query=НЕ...
Name: 156, dtype: object

## Find kinopoisk id over serch through kinopoisk site

In [3]:
class KinopoiskSearchSpider(scrapy.Spider):
    name = "yandex.help"
    
    custom_settings = {
        'DOWNLOAD_DELAY': 7,
        'DOWNLOAD_TIMEOUT' : 30,
        'RETRY_ENABLED': False,
        #'HTTPPROXY_ENABLED': True
    }
    
    def __init__(self, search_query_urls):
        super().__init__()
        
        self.search_query_urls = search_query_urls
        
    def start_requests(self):
        
        urls = self.search_query_urls

        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
       
        project_title = response.url.split('=')[1]
        project_id_kinopoisk = response.xpath("//div[@class='element most_wanted']//@data-id").extract_first()
        scraped_info = {
            'project_title': project_title,
            'project_id_kinopoisk': project_id_kinopoisk
        }
        yield scraped_info

In [ ]:
process = CrawlerProcess(settings={
    'FEED_FORMAT': 'csv',
    'FEED_URI': 'kinopoisk_search.csv'
})
process.crawl(KinopoiskSearchSpider, search_query_urls=project_id_title['search_query'].values.tolist())
process.start()

In [16]:
temp_data = pd.read_csv('kinopoisk_search.csv')
temp_data['project_id_kinopoisk'] = temp_data['project_id_kinopoisk'].fillna(-1)
temp_data['project_title_rus'] = temp_data['project_title'].apply(lambda x: unquote(x))

In [29]:
project_id_title.head()

,project_id,project_title,search_query
0,30,Экстрасенсы ведут расследование,https://www.kinopoisk.ru/index.php?kp_query=Эк...
1,31,Полицейский с Рублёвки,https://www.kinopoisk.ru/index.php?kp_query=По...
2,32,Школа ремонта,https://www.kinopoisk.ru/index.php?kp_query=Шк...
3,33,Ольга,https://www.kinopoisk.ru/index.php?kp_query=Ольга
4,34,Адаптация,https://www.kinopoisk.ru/index.php?kp_query=Ад...


In [58]:
project_info = project_id_title.merge(temp_data[['project_title_rus', 'project_id_kinopoisk']], 
                       how='left', left_on=['project_title'], right_on=['project_title_rus'])
project_info = project_info[['project_id', 'project_title', 'search_query', 'project_id_kinopoisk']].copy()
project_info.head()

,project_id,project_title,search_query,project_id_kinopoisk
0,30,Экстрасенсы ведут расследование,https://www.kinopoisk.ru/index.php?kp_query=Эк...,927654.0
1,31,Полицейский с Рублёвки,https://www.kinopoisk.ru/index.php?kp_query=По...,913033.0
2,32,Школа ремонта,https://www.kinopoisk.ru/index.php?kp_query=Шк...,916853.0
3,33,Ольга,https://www.kinopoisk.ru/index.php?kp_query=Ольга,940680.0
4,34,Адаптация,https://www.kinopoisk.ru/index.php?kp_query=Ад...,584.0


In [59]:
project_info[project_info['project_id_kinopoisk'].isnull()]

,project_id,project_title,search_query,project_id_kinopoisk
28,61,САШАТАНЯ,https://www.kinopoisk.ru/index.php?kp_query=СА...,NaN
34,72,Филфак,https://www.kinopoisk.ru/index.php?kp_query=Фи...,NaN
44,87,ХБ,https://www.kinopoisk.ru/index.php?kp_query=ХБ,NaN
54,97,Неzлоб,https://www.kinopoisk.ru/index.php?kp_query=Не...,NaN
76,120,Ху из ху,https://www.kinopoisk.ru/index.php?kp_query=Ху...,NaN
80,125,Бешенл Джеографик,https://www.kinopoisk.ru/index.php?kp_query=Бе...,NaN


In [67]:
project_info2 = project_info[~project_info['project_id_kinopoisk'].isnull()].copy()
project_info2['project_id_kinopoisk'] = project_info2['project_id_kinopoisk'].astype(int)

In [69]:
cast_str = 'https://www.kinopoisk.ru/film/{kinopoisk_id}/cast/'
project_info2['cast_url'] = project_info2['project_id_kinopoisk'].apply(lambda x: 
                                                                        cast_str.format(kinopoisk_id=int(x)))

In [73]:
project_info2.to_csv('project_info2.csv', index=False, header=True)

## Scrap cast teams from kinopoisk

In [25]:
project_info2 = pd.read_csv('project_info2.csv')
project_info2.head(2)

,project_id,project_title,search_query,project_id_kinopoisk,cast_url
0,30,Экстрасенсы ведут расследование,https://www.kinopoisk.ru/index.php?kp_query=Эк...,927654,https://www.kinopoisk.ru/film/927654/cast/
1,31,Полицейский с Рублёвки,https://www.kinopoisk.ru/index.php?kp_query=По...,913033,https://www.kinopoisk.ru/film/913033/cast/


In [4]:
class KinopoiskSpider(scrapy.Spider):
    name = "yandex.help"
    custom_settings = {
        'DOWNLOAD_DELAY': 7,
        'DOWNLOAD_TIMEOUT' : 30,
        'RETRY_ENABLED': False,
        #'HTTPPROXY_ENABLED': True
    }
    
    def __init__(self, cast_urls):
        super().__init__()
        
        self.cast_urls = cast_urls
        
    def start_requests(self):
        
        urls = self.cast_urls
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        project_id_kinopoisk = response.url.split("/")[-3]
        
        #all_names = response.xpath("//div[@class='name']//text()").extract()
        #roles = response.xpath("//div[@class='dub no_dub dub_first']/preceding-sibling::*[1]//text()").extract()
        #first_role_name = response.xpath("//div[@class='dub no_dub dub_first']//div[@class='name']//text()").extract()       
        #all_info = response.xpath("normalize-space(//div[@class='block_left']//text())").extract()
        all_info = response.xpath("normalize-space(//div[@class='block_left'])").extract()
        
        scraped_info = {
            'project_id_kinopoisk': project_id_kinopoisk,
            'all_info': all_info
            #'all_names': all_names,
            #'roles': roles,
            #'first_role_name': first_role_name,
        }
        yield scraped_info

In [5]:
urls_to_parse = ['https://www.kinopoisk.ru/film/1260090/cast/']

In [ ]:
process = CrawlerProcess(settings={
    'FEED_FORMAT': 'csv',
    'FEED_URI':'data_files/kinopoisk_cast_2019_08_14_13.csv',
    #'FEED_URI': 'kinopoisk_cast.csv'
})
process.crawl(KinopoiskSpider, cast_urls=urls_to_parse)#project_info2['cast_url'].values.tolist())
process.start()

In [2]:
kinopoisk_cast = pd.read_csv('kinopoisk_cast.csv')
kinopoisk_cast.head()

,page,all_info
0,913033,var AncherStart = {}; $(function(){ $('div.dub...
1,584,var AncherStart = {}; $(function(){ $('div.dub...
2,630165,var AncherStart = {}; $(function(){ $('div.dub...
3,927654,var AncherStart = {}; $(function(){ $('div.dub...
4,1049171,var AncherStart = {}; $(function(){ $('div.dub...


In [6]:
current_str = kinopoisk_cast['all_info'].iloc[0]

In [8]:
    old_key_words = ['режиссеры', 'актеры', 'продюсеры',
                     'режиссеры дубляжа', 'переводчики', 'актеры дубляжа',
                     'сценаристы', 'операторы','композиторы',
                     'художники', 'монтажеры']
    key_words = ['режиссер', 'актер', 'продюсер', 
                 'режиссер дубляжа', 'переводчик', 'актер дубляжа',
                 'сценарист', 'оператор','композитор',
                 'художник', 'монтажер']
    replace_key_words = dict(zip(old_key_words, key_words))
    temp_str = current_str.lower()
    
    cut_point_re = re.compile(r'режиссер 1')
    search_res = re.search(cut_point_re, temp_str)
    if search_res is not None:
        cut_point = search_res.span()[0]
        temp_str = temp_str[cut_point:]
    
    for word in replace_key_words:
        temp_str = temp_str.replace(word, replace_key_words[word])

In [9]:
    # start and end indicies of keyword  
    key_word_inds = {}

    for word in key_words:
        temp_re = re.compile(word + ' 1')
        inds_res = re.search(temp_re, temp_str)
        if inds_res is not None:
            key_word_inds[word] = inds_res.span()

    key_word_inds_sorted = sorted(key_word_inds.items(), key=lambda kv: kv[1][0])  

In [10]:
key_word_inds_sorted

[('режиссер', (0, 10)),
 ('актер', (27, 34)),
 ('продюсер', (2517, 2527)),
 ('актер дубляжа', (2625, 2640)),
 ('сценарист', (2665, 2676)),
 ('оператор', (2693, 2703)),
 ('композитор', (2756, 2768)),
 ('художник', (2790, 2800)),
 ('монтажер', (2912, 2922))]

In [17]:
temp_str[2912:2922]

'монтажер 1'

In [18]:
temp_str[2921:]

'1. степан гордеев \xa0 2. игорь абидов \xa0'

In [19]:
len(temp_str)

2958

In [13]:
len(key_word_inds_sorted)

9

In [15]:
key_word_inds_sorted[8]

('монтажер', (2912, 2922))

In [20]:
    key_word_list = {}
    person_re = re.compile(r'\d+\. \w+ \w+')

    for ind in range(len(key_word_inds_sorted)):
        if ind > 0:
            #print(ind-1, ind)
            kw = key_word_inds_sorted[ind-1][0]
            start_ind = key_word_inds_sorted[ind-1][1][1]-1
            end_ind = key_word_inds_sorted[ind][1][0]-1
            #print(kw, start_ind, end_ind)
            
            temp_persons = re.findall(person_re, temp_str[start_ind:end_ind])
            temp_persons = [p.split('. ')[1] for p in temp_persons]
            key_word_list[kw] = temp_persons
        if ind == len(key_word_inds_sorted)-1:
            kw = key_word_inds_sorted[ind][0]
            start_ind = key_word_inds_sorted[ind][1][1]-1
            
            temp_persons = re.findall(person_re, temp_str[start_ind:])
            temp_persons = [p.split('. ')[1] for p in temp_persons]
            key_word_list[kw] = temp_persons                      

In [21]:
key_word_list

{'режиссер': ['илья куликов'],
 'актер': ['александр петров',
  'сергей бурунов',
  'татьяна бабенкова',
  'софия каштанова',
  'роман попов',
  'александра бортич',
  'анна котова',
  'ростислав гулбис',
  'сергей штатнов',
  'сергей журавлев',
  'владимир бутенко',
  'мария староторжская',
  'лилия лаврова',
  'анастасия стежко',
  'виктория тарасова',
  'сергей калашников',
  'евгений левочкин',
  'николай соловьев',
  'карина реука',
  'михаил шамигулов',
  'николай лунин',
  'татьяна мошкова',
  'илья лукашенко',
  'владимир рузанов',
  'михаил богдасаров',
  'константин тополага',
  'светлана суханова',
  'наиль абдрахманов',
  'маруся климова',
  'павел галич',
  'иван кравченко',
  'ольга вербицкая',
  'юрий брешин',
  'александр воробьев',
  'владимир колида',
  'андрей лебедев',
  'татьяна филатова',
  'максим костромыкин',
  'павел чукреев',
  'светлана нехороших',
  'василий шемякинский',
  'сергей крапивенцев',
  'дарья погодина',
  'александра виноградова',
  'игорь шарой

In [28]:
def parse_str_to_persons(current_str):

    old_key_words = ['режиссеры', 'актеры', 'продюсеры',
                     'режиссеры дубляжа', 'переводчики', 'актеры дубляжа',
                     'сценаристы', 'операторы','композиторы',
                     'художники', 'художники-постановщики', 'монтажеры']
    key_words = ['режиссер', 'актер', 'продюсер', 
                 'режиссер дубляжа', 'переводчик', 'актер дубляжа',
                 'сценарист', 'оператор','композитор',
                 'художник', 'художник-постановщик', 'монтажер']
    replace_key_words = dict(zip(old_key_words, key_words))
    temp_str = current_str.lower()
    
    # cut header
    cut_point_re = re.compile(r'режиссер 1')
    search_res = re.search(cut_point_re, temp_str)
    if search_res is not None:
        cut_point = search_res.span()[0]
        temp_str = temp_str[cut_point:]
    
    # change key words
    for word in replace_key_words:
        temp_str = temp_str.replace(word, replace_key_words[word])

    # start and end indicies of keyword  
    key_word_inds = {}
    for word in key_words:
        temp_re = re.compile(word + ' 1')
        inds_res = re.search(temp_re, temp_str)
        if inds_res is not None:
            key_word_inds[word] = inds_res.span()

    key_word_inds_sorted = sorted(key_word_inds.items(), key=lambda kv: kv[1][0])   
    key_word_list = {}
    person_re = re.compile(r'\d+\. \w+ \w+')
    # get persons from string
    for ind in range(len(key_word_inds_sorted)):
        if ind > 0:
            kw = key_word_inds_sorted[ind-1][0]
            start_ind = key_word_inds_sorted[ind-1][1][1]-1
            end_ind = key_word_inds_sorted[ind][1][0]-1
            # cut part of string
            temp_persons = re.findall(person_re, temp_str[start_ind:end_ind])
            temp_persons = [p.split('. ')[1] for p in temp_persons]
            key_word_list[kw] = temp_persons
        if ind == len(key_word_inds_sorted)-1:
            kw = key_word_inds_sorted[ind][0]
            start_ind = key_word_inds_sorted[ind][1][1]-1
            # cut part of string
            temp_persons = re.findall(person_re, temp_str[start_ind:])
            temp_persons = [p.split('. ')[1] for p in temp_persons]
            key_word_list[kw] = temp_persons  
    # transform into list     
    person_role = []
    for role in key_word_list:
        for person in key_word_list[role]:
            person_role.append((person, role))
            
    return person_role

In [29]:
kinopoisk_cast['person_role_list'] = kinopoisk_cast['all_info'].apply(lambda x: parse_str_to_persons(x))
kinopoisk_cast = kinopoisk_cast.rename(columns={'page':'project_id_kinopoisk'})

In [30]:
kinopoisk_cast.head()

,project_id_kinopoisk,all_info,person_role_list
0,913033,var AncherStart = {}; $(function(){ $('div.dub...,"[(илья куликов, режиссер), (александр петров, ..."
1,584,var AncherStart = {}; $(function(){ $('div.dub...,"[(спайк джонс, режиссер), (николас кейдж, акте..."
2,630165,var AncherStart = {}; $(function(){ $('div.dub...,"[(константин смирнов, режиссер), (максим зыков..."
3,927654,var AncherStart = {}; $(function(){ $('div.dub...,"[(мария шайкевич, продюсер), (владимир спиринк..."
4,1049171,var AncherStart = {}; $(function(){ $('div.dub...,"[(жанна кадникова, режиссер), (николай бурлак,..."


In [31]:
result_table = project_info2.merge(kinopoisk_cast[['project_id_kinopoisk', 'person_role_list']],
                    how='left', on=['project_id_kinopoisk'])
result_table.head()

,project_id,project_title,search_query,project_id_kinopoisk,cast_url,person_role_list
0,30,Экстрасенсы ведут расследование,https://www.kinopoisk.ru/index.php?kp_query=Эк...,927654,https://www.kinopoisk.ru/film/927654/cast/,"[(мария шайкевич, продюсер), (владимир спиринк..."
1,31,Полицейский с Рублёвки,https://www.kinopoisk.ru/index.php?kp_query=По...,913033,https://www.kinopoisk.ru/film/913033/cast/,"[(илья куликов, режиссер), (александр петров, ..."
2,32,Школа ремонта,https://www.kinopoisk.ru/index.php?kp_query=Шк...,916853,https://www.kinopoisk.ru/film/916853/cast/,"[(владимир чайченко, режиссер), (александр гри..."
3,33,Ольга,https://www.kinopoisk.ru/index.php?kp_query=Ольга,940680,https://www.kinopoisk.ru/film/940680/cast/,"[(алексей нужный, режиссер), (игорь волошин, р..."
4,34,Адаптация,https://www.kinopoisk.ru/index.php?kp_query=Ад...,584,https://www.kinopoisk.ru/film/584/cast/,"[(спайк джонс, режиссер), (николас кейдж, акте..."


In [32]:
result_table.to_csv('parsing_result_2019_07_26.csv', index=False, header=True)